In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [67]:
#計測地点のosm_idを指定
start_osm = 228330073
mid_osm = [228330071,38135912,52171227,52171226,52171225]
end_osm = 315917801
start_mesh = 50325643134
end_mesh = 50325623342

In [68]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [69]:
getdbname = "kddi_06_new_param"
dbname = "kddi_trkn06_monthly_anbun_2f_14"
trknName = "trkn_06_grp"
sttime = "2022-04-01 06:00:00"

In [70]:
datedir = "230225"
f = open(datedir+'/'+ dbname + '.csv', 'w')
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        #cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        #cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        #cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            #cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            #cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        #cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        #cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                #print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #cur.execute(sql)
                #connection.commit()

        #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
        #cur.execute(sql)
        #connection.commit()
        #cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)
f.close()

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値5.366666666666667距離0.0 km
存在メッシュ：50325643123値5.766232340775622距離0.2726531182505025 km
存在メッシュ：50325643134値5.366666666666667距離0.0 km
存在メッシュ：50325643141値5.638040059798209距離0.18517807369848877 km
存在メッシュ：50325643132値5.535987549681474距離0.11554012201340155 km
存在メッシュ：50325643121値5.917122018427693距離0.3756162699661618 km
存在メッシュ：50325643123値5.766232340775622距離0.2726531182505025 km
存在メッシュ：50325633343値6.076376784154322距離0.4842875380085171 km
存在メッシュ：50325633342値6.313579138432099距離0.6461481925665478 km
存在メッシュ：50325633341値6.23942942001655距離0.595550372850224 km
存在メッシュ：50325633344値6.165800712290389距離0.5453080771398616 km
存在メッシュ：50325633324値6.467580100002558距離0.7512344026855676 km
存在メッシュ：50325633342値6.313579138432099距離0.6461481925665478 km
存在メッシュ：50325633341値6.23942942001655距離0.595550372850224 km
存在メッシュ：50325633324値6.467580100002558距離0.7512344026855676 km
存在メッシュ：50325633322値6.625521847608622距離0.8590097016592841 km
存在メッシュ：503256333

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値29.216666666666665距離0.0 km
存在メッシュ：50325643123値32.01808250818893距離0.2726531182505025 km
存在メッシュ：50325643134値29.216666666666665距離0.0 km
50325643141 is kddi none!
50325643132 is kddi none!
存在メッシュ：50325643121値33.07599303626255距離0.3756162699661618 km
存在メッシュ：50325643123値32.01808250818893距離0.2726531182505025 km
存在メッシュ：50325633343値34.19255247180316距離0.4842875380085171 km
存在メッシュ：50325633342値35.85561433116334距離0.6461481925665478 km
存在メッシュ：50325633341値35.335739353721976距離0.595550372850224 km
存在メッシュ：50325633344値34.81951726163447距離0.5453080771398616 km
存在メッシュ：50325633324値36.93533854499934距離0.7512344026855676 km
存在メッシュ：50325633342値35.85561433116334距離0.6461481925665478 km
存在メッシュ：50325633341値35.335739353721976距離0.595550372850224 km
存在メッシュ：50325633324値36.93533854499934距離0.7512344026855676 km
50325633322 is kddi none!
存在メッシュ：50325633342値35.85561433116334距離0.6461481925665478 km
50325633122 is kddi none!
存在メッシュ：50325623344値43.769377

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値27.716666666666665距離0.0 km
存在メッシュ：50325643123値30.027909004635195距離0.2726531182505025 km
存在メッシュ：50325643134値27.716666666666665距離0.0 km
存在メッシュ：50325643141値29.286395290133875距離0.18517807369848877 km
存在メッシュ：50325643132値28.696084116373132距離0.11554012201340155 km
存在メッシュ：50325643121値30.90071323670442距離0.3756162699661618 km
存在メッシュ：50325643123値30.027909004635195距離0.2726531182505025 km
存在メッシュ：50325633343値31.821904372282987距離0.4842875380085171 km
存在メッシュ：50325633342値33.19397449591206距離0.6461481925665478 km
存在メッシュ：50325633341値32.765063857047394距離0.595550372850224 km
存在メッシュ：50325633344値32.3391669454418距離0.5453080771398616 km
存在メッシュ：50325633324値34.08477559704082距離0.7512344026855676 km
存在メッシュ：50325633342値33.19397449591206距離0.6461481925665478 km
存在メッシュ：50325633341値32.765063857047394距離0.595550372850224 km
存在メッシュ：50325633324値34.08477559704082距離0.7512344026855676 km
存在メッシュ：50325633322値34.998371728174774距離0.8590097016592841 km
存在メッシ

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134 is kddi none!
存在メッシュ：50325643123値35.86399953445591距離0.2726531182505025 km
50325643134 is kddi none!
存在メッシュ：50325643141値34.875632282607754距離0.18517807369848877 km
存在メッシュ：50325643132値34.08880363583411距離0.11554012201340155 km
存在メッシュ：50325643121値37.02736455843507距離0.3756162699661618 km
存在メッシュ：50325643123値35.86399953445591距離0.2726531182505025 km
存在メッシュ：50325633343値38.25522472243889距離0.4842875380085171 km
存在メッシュ：50325633342値40.08406369185194距離0.6461481925665478 km
存在メッシュ：50325633341値39.51236660637294距離0.595550372850224 km
存在メッシュ：50325633344値38.944686532677565距離0.5453080771398616 km
存在メッシュ：50325633324値41.271416830502986距離0.7512344026855676 km
存在メッシュ：50325633342値40.08406369185194距離0.6461481925665478 km
存在メッシュ：50325633341値39.51236660637294距離0.595550372850224 km
存在メッシュ：50325633324値41.271416830502986距離0.7512344026855676 km
存在メッシュ：50325633322値42.48915357598617距離0.8590097016592841 km
存在メッシュ：50325633342値40.08406369185194距離0.64614819

50325643134 is kddi none!
存在メッシュ：50325643123値37.24783816089979距離0.2726531182505025 km
50325643134 is kddi none!
存在メッシュ：50325643141値36.18979583260704距離0.18517807369848877 km
50325643132 is kddi none!
存在メッシュ：50325643121値38.493214751760725距離0.3756162699661618 km
存在メッシュ：50325643123値37.24783816089979距離0.2726531182505025 km
存在メッシュ：50325633343値39.807633075831326距離0.4842875380085171 km
存在メッシュ：50325633342値41.76539628343662距離0.6461481925665478 km
存在メッシュ：50325633341値41.153397338441934距離0.595550372850224 km
存在メッシュ：50325633344値40.54569858499282距離0.5453080771398616 km
存在メッシュ：50325633324値43.036452034196216距離0.7512344026855676 km
存在メッシュ：50325633342値41.76539628343662距離0.6461481925665478 km
存在メッシュ：50325633341値41.153397338441934距離0.595550372850224 km
存在メッシュ：50325633324値43.036452034196216距離0.7512344026855676 km
存在メッシュ：50325633322値44.3400332880565距離0.8590097016592841 km
存在メッシュ：50325633342値41.76539628343662距離0.6461481925665478 km
存在メッシュ：50325633122値49.716086630624915距離1.303482000243029 km
存在メッシュ：50325623344

...

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値5.366666666666667距離0.0 km
存在メッシュ：50325643123値5.766232340775622距離0.2726531182505025 km
存在メッシュ：50325643134値5.366666666666667距離0.0 km
存在メッシュ：50325643141値5.638040059798209距離0.18517807369848877 km
50325643132 is kddi none!
存在メッシュ：50325643121値5.917122018427693距離0.3756162699661618 km
存在メッシュ：50325643123値5.766232340775622距離0.2726531182505025 km
存在メッシュ：50325633343値6.076376784154322距離0.4842875380085171 km
存在メッシュ：50325633342値6.313579138432099距離0.6461481925665478 km
存在メッシュ：50325633341値6.23942942001655距離0.595550372850224 km
存在メッシュ：50325633344値6.165800712290389距離0.5453080771398616 km
存在メッシュ：50325633324値6.467580100002558距離0.7512344026855676 km
存在メッシュ：50325633342値6.313579138432099距離0.6461481925665478 km
存在メッシュ：50325633341値6.23942942001655距離0.595550372850224 km
存在メッシュ：50325633324値6.467580100002558距離0.7512344026855676 km
存在メッシュ：50325633322値6.625521847608622距離0.8590097016592841 km
存在メッシュ：50325633342値6.313579138432099距離0.64614819256

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値29.216666666666665距離0.0 km
存在メッシュ：50325643123値32.01808250818893距離0.2726531182505025 km
存在メッシュ：50325643134値29.216666666666665距離0.0 km
50325643141 is kddi none!
50325643132 is kddi none!
存在メッシュ：50325643121値33.07599303626255距離0.3756162699661618 km
存在メッシュ：50325643123値32.01808250818893距離0.2726531182505025 km
50325633343 is kddi none!
存在メッシュ：50325633342値35.85561433116334距離0.6461481925665478 km
存在メッシュ：50325633341値35.335739353721976距離0.595550372850224 km
存在メッシュ：50325633344値34.81951726163447距離0.5453080771398616 km
存在メッシュ：50325633324値36.93533854499934距離0.7512344026855676 km
存在メッシュ：50325633342値35.85561433116334距離0.6461481925665478 km
存在メッシュ：50325633341値35.335739353721976距離0.595550372850224 km
存在メッシュ：50325633324値36.93533854499934距離0.7512344026855676 km
存在メッシュ：50325633322値38.04269221037123距離0.8590097016592841 km
存在メッシュ：50325633342値35.85561433116334距離0.6461481925665478 km
存在メッシュ：50325633122値42.609490295645距離1.303482000243029 

存在メッシュ：50325643134値28.083333333333332距離0.0 km
存在メッシュ：50325643123値30.36703330803079距離0.2726531182505025 km
存在メッシュ：50325643134値28.083333333333332距離0.0 km
存在メッシュ：50325643141値29.634355984140605距離0.18517807369848877 km
50325643132 is kddi none!
存在メッシュ：50325643121値31.22943660081759距離0.3756162699661618 km
存在メッシュ：50325643123値30.36703330803079距離0.2726531182505025 km
存在メッシュ：50325633343値32.139650185283756距離0.4842875380085171 km
存在メッシュ：50325633342値33.495369771733074距離0.6461481925665478 km
存在メッシュ：50325633341値33.07157032888294距離0.595550372850224 km
存在メッシュ：50325633344値32.6507486996321距離0.5453080771398616 km
存在メッシュ：50325633324値34.37555547043236距離0.7512344026855676 km
存在メッシュ：50325633342値33.495369771733074距離0.6461481925665478 km
存在メッシュ：50325633341値33.07157032888294距離0.595550372850224 km
存在メッシュ：50325633324値34.37555547043236距離0.7512344026855676 km
50325633322 is kddi none!
存在メッシュ：50325633342値33.495369771733074距離0.6461481925665478 km
存在メッシュ：50325633122値39.00109517693224距離1.303482000243029 km
存在メッシュ：5032562

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134 is kddi none!
存在メッシュ：50325643123値35.86399953445591距離0.2726531182505025 km
50325643134 is kddi none!
存在メッシュ：50325643141値34.875632282607754距離0.18517807369848877 km
50325643132 is kddi none!
存在メッシュ：50325643121値37.02736455843507距離0.3756162699661618 km
存在メッシュ：50325643123値35.86399953445591距離0.2726531182505025 km
存在メッシュ：50325633343値38.25522472243889距離0.4842875380085171 km
存在メッシュ：50325633342値40.08406369185194距離0.6461481925665478 km
存在メッシュ：50325633341値39.51236660637294距離0.595550372850224 km
存在メッシュ：50325633344値38.944686532677565距離0.5453080771398616 km
存在メッシュ：50325633324値41.271416830502986距離0.7512344026855676 km
存在メッシュ：50325633342値40.08406369185194距離0.6461481925665478 km
存在メッシュ：50325633341値39.51236660637294距離0.595550372850224 km
存在メッシュ：50325633324値41.271416830502986距離0.7512344026855676 km
50325633322 is kddi none!
存在メッシュ：50325633342値40.08406369185194距離0.6461481925665478 km
存在メッシュ：50325633122値47.51117861779837距離1.303482000243029 k

33.7734375000:33.7869791665:132.789843750:132.789843750
50325643134 is kddi none!
存在メッシュ：50325643123値37.226735602544075距離0.2726531182505025 km
50325643134 is kddi none!
存在メッシュ：50325643141値36.17546358828321距離0.18517807369848877 km
存在メッシュ：50325643132値35.338557129853776距離0.11554012201340155 km
存在メッシュ：50325643121値38.464143144924996距離0.3756162699661618 km
存在メッシュ：50325643123値37.226735602544075距離0.2726531182505025 km
存在メッシュ：50325633343値39.77015062891363距離0.4842875380085171 km
存在メッシュ：50325633342値41.71538629261912距離0.6461481925665478 km
存在メッシュ：50325633341値41.10730347170945距離0.595550372850224 km
存在メッシュ：50325633344値40.50349332582131距離0.5453080771398616 km
存在メッシュ：50325633324値42.97830867635307距離0.7512344026855676 km
存在メッシュ：50325633342値41.71538629261912距離0.6461481925665478 km
存在メッシュ：50325633341値41.10730347170945距離0.595550372850224 km
存在メッシュ：50325633324値42.97830867635307距離0.7512344026855676 km
存在メッシュ：50325633322値44.27354843553143距離0.8590097016592841 km
存在メッシュ：50325633342値41.71538629261912距離0.64614819

In [71]:
connection.close()

In [72]:
#CreateGeojsonFile